In [ ]:
!pip install awscli


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.2/548.2 kB 33.0 MB/s eta 0:00:00
  Attempting uninstall: rsa
    Found existing installation: rsa 4.9
    Uninstalling rsa-4.9:
      Successfully uninstalled rsa-4.9
  Attempting uninstall: docutils
    Found existing installation: docutils 0.21.2
    Uninstalling docutils-0.21.2:
      Successfully uninstalled docutils-0.21.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sphinx 8.1.3 requires docutils<0.22,>=0.20, but you have docutils 0.16 which is incompatible.


In [ ]:
!aws configure


In [ ]:
!pip install boto3


In [ ]:
import boto3

# Cấu hình trực tiếp
session = boto3.Session(
    aws_access_key_id='AWS Access Key ID',
    aws_secret_access_key='AWS Secret Access Key',
    region_name='ap-southeast-2'
)

# Tạo resource S3
s3 = session.resource('s3')

# Tải file từ S3
bucket_name = "bigdata-ecommerce"
file_key = "E-commerceCustomerBehavior.csv"
obj = s3.Object(bucket_name, file_key)
content = obj.get()['Body'].read().decode('utf-8')

print(content)


Nội dung file:
Customer ID,Gender,Age,City,Membership Type,Total Spend,Items Purchased,Average Rating,Discount Applied,Days Since Last Purchase,Satisfaction Level
101,Female,29,New York,Gold,1120.2,14,4.6,TRUE,25,Satisfied
102,Male,34,Los Angeles,Silver,780.5,11,4.1,FALSE,18,Neutral
103,Female,43,Chicago,Bronze,510.75,9,3.4,TRUE,42,Unsatisfied
104,Male,30,San Francisco,Gold,1480.3,19,4.7,FALSE,12,Satisfied
105,Male,27,Miami,Silver,720.4,13,4,TRUE,55,Unsatisfied
106,Female,37,Houston,Bronze,440.8,8,3.1,FALSE,22,Neutral
107,Female,31,New York,Gold,1150.6,15,4.5,TRUE,28,Satisfied
108,Male,35,Los Angeles,Silver,800.9,12,4.2,FALSE,14,Neutral
109,Female,41,Chicago,Bronze,495.25,10,3.6,TRUE,40,Unsatisfied
110,Male,28,San Francisco,Gold,1520.1,21,4.8,FALSE,9,Satisfied
111,Male,32,Miami,Silver,690.3,11,3.8,TRUE,34,Unsatisfied
112,Female,36,Houston,Bronze,470.5,7,3.2,FALSE,20,Neutral
113,Female,30,New York,Gold,1200.8,16,4.3,TRUE,21,Satisfied
114,Male,33,Los Angeles,Silver,820.75,13,4.4,FALSE,15

In [ ]:
import pandas as pd
from pyspark.sql import SparkSession
from io import StringIO

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("PySpark Example with S3") \
    .getOrCreate()

# Chuyển dữ liệu từ content (string) thành Pandas DataFrame
data = StringIO(content)
df_pandas = pd.read_csv(data)

# Chuyển từ Pandas DataFrame sang PySpark DataFrame
df_spark = spark.createDataFrame(df_pandas)

# Hiển thị 5 dòng đầu tiên
df_spark.show(5)


+-----------+------+---+-------------+---------------+-----------+---------------+--------------+----------------+------------------------+------------------+
|Customer ID|Gender|Age|         City|Membership Type|Total Spend|Items Purchased|Average Rating|Discount Applied|Days Since Last Purchase|Satisfaction Level|
+-----------+------+---+-------------+---------------+-----------+---------------+--------------+----------------+------------------------+------------------+
|        101|Female| 29|     New York|           Gold|     1120.2|             14|           4.6|            true|                      25|         Satisfied|
|        102|  Male| 34|  Los Angeles|         Silver|      780.5|             11|           4.1|           false|                      18|           Neutral|
|        103|Female| 43|      Chicago|         Bronze|     510.75|              9|           3.4|            true|                      42|       Unsatisfied|
|        104|  Male| 30|San Francisco|        

In [ ]:
# Thực hiện một số thao tác cơ bản với PySpark

df_filtered = df_spark.filter(df_spark["Age"] < 30)
df_filtered.show(5)

# Tính tổng số lượng dòng
count = df_filtered.count()
print(f"Số khách hàng nhỏ hơn 30 tuổi: {count}")


+-----------+------+---+-------------+---------------+-----------+---------------+--------------+----------------+------------------------+------------------+
|Customer ID|Gender|Age|         City|Membership Type|Total Spend|Items Purchased|Average Rating|Discount Applied|Days Since Last Purchase|Satisfaction Level|
+-----------+------+---+-------------+---------------+-----------+---------------+--------------+----------------+------------------------+------------------+
|        101|Female| 29|     New York|           Gold|     1120.2|             14|           4.6|            true|                      25|         Satisfied|
|        105|  Male| 27|        Miami|         Silver|      720.4|             13|           4.0|            true|                      55|       Unsatisfied|
|        110|  Male| 28|San Francisco|           Gold|     1520.1|             21|           4.8|           false|                       9|         Satisfied|
|        116|  Male| 29|San Francisco|        

In [ ]:
# Dừng SparkSession khi hoàn thành
spark.stop()


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when

spark = SparkSession.builder.appName("EcommerceDataProcessing").getOrCreate()
`
import io
import pandas as pd

csv_buffer = io.StringIO(content)
pandas_df = pd.read_csv(csv_buffer)

df_spark = spark.createDataFrame(pandas_df)

df_spark_age_groups = df_spark.withColumn(
    "Age Group",
    when(df_spark["Age"] < 35, "Under 35")
    .when((df_spark["Age"] >= 35) & (df_spark["Age"] <= 50), "Between 35-50")
    .otherwise("Over 50")
)

customer_segments = df_spark_age_groups.groupBy("Membership Type", "Age Group", "City").agg(
    {"Total Spend": "mean", "Items Purchased": "sum"}
)

city_segments = df_spark_age_groups.groupBy("City").agg(
    {"Total Spend": "mean", "Items Purchased": "sum"}
)

at_risk_customers = df_spark.filter(
    (df_spark["Days Since Last Purchase"] > 30) &
    (df_spark["Satisfaction Level"] == "Unsatisfied")
)

# Hiển thị kết quả
print('Customer Segmentation Result:')
customer_segments.show()

print('City Segmentation Result:')
city_segments.show()

print('Customers at Risk Result:')
at_risk_customers.show()


Customer Segmentation Result:
+---------------+-------------+-------------+--------------------+------------------+
|Membership Type|    Age Group|         City|sum(Items Purchased)|  avg(Total Spend)|
+---------------+-------------+-------------+--------------------+------------------+
|         Silver|Between 35-50|  Los Angeles|                 288| 814.6499999999997|
|         Bronze|Between 35-50|      Chicago|                 546|499.88275862068963|
|           Gold|     Under 35|     New York|                 901|1165.0355932203386|
|         Silver|     Under 35|  Los Angeles|                 401| 799.2114285714288|
|         Bronze|Between 35-50|      Houston|                 439| 446.8948275862068|
|         Silver|     Under 35|        Miami|                 675| 690.3896551724137|
|           Gold|     Under 35|San Francisco|                1141|1460.4561403508767|
|           Gold|Between 35-50|San Francisco|                  19|            1420.8|
+---------------+-------

In [ ]:
import boto3

dynamodb = boto3.resource('dynamodb', region_name='ap-southeast-2')

customer_segments_table = dynamodb.create_table(
    TableName='CustomerSegments',
    KeySchema=[
        {'AttributeName': 'MembershipType', 'KeyType': 'HASH'},  # Partition key
        {'AttributeName': 'City', 'KeyType': 'RANGE'}  # Sort key
    ],
    AttributeDefinitions=[
        {'AttributeName': 'MembershipType', 'AttributeType': 'S'},  # String
        {'AttributeName': 'City', 'AttributeType': 'S'}  # String
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
)

city_segments_table = dynamodb.create_table(
    TableName='CitySegments',
    KeySchema=[
        {'AttributeName': 'City', 'KeyType': 'HASH'}  # Partition key
    ],
    AttributeDefinitions=[
        {'AttributeName': 'City', 'AttributeType': 'S'}  # String
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
)

at_risk_customers_table = dynamodb.create_table(
    TableName='AtRiskCustomers',
    KeySchema=[
        {'AttributeName': 'CustomerID', 'KeyType': 'HASH'}  # Partition key
    ],
    AttributeDefinitions=[
        {'AttributeName': 'CustomerID', 'AttributeType': 'S'}  # String
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
)

customer_segments_table.meta.client.get_waiter('table_exists').wait(TableName='CustomerSegments')
city_segments_table.meta.client.get_waiter('table_exists').wait(TableName='CitySegments')
at_risk_customers_table.meta.client.get_waiter('table_exists').wait(TableName='AtRiskCustomers')



In [ ]:
import boto3
import csv

# Cấu hình trực tiếp
session = boto3.Session(
    aws_access_key_id='AWS Access Key ID',
    aws_secret_access_key='AWS Secret Access Key',
    region_name='ap-southeast-2'
)

# Tạo resource DynamoDB và S3, sử dụng session đã cấu hình
dynamodb = session.resource('dynamodb') # Using the session object for region
s3 = session.resource('s3') # Using the session object for region


# Tên bucket và key của file
bucket_name = "bigdata-ecommerce"
file_key = "E-commerceCustomerBehavior.csv"

# Tải file từ S3
obj = s3.Object(bucket_name, file_key)
content = obj.get()['Body'].read().decode('utf-8')

# Phân tích nội dung CSV
csv_reader = csv.DictReader(content.splitlines())

# Tên bảng DynamoDB
table_name = "big-data"

# Lấy bảng DynamoDB
table = dynamodb.Table(table_name)

# Lặp qua từng dòng trong file CSV và thêm vào DynamoDB
for row in csv_reader:
    # Lưu từng dòng vào DynamoDB
    table.put_item(Item=row)

print("Du lieu da duoc luu")

Du lieu da duoc luu


In [ ]:
import boto3
from pyspark.sql.functions import to_json
import json
from decimal import Decimal

# Kết nối DynamoDB
dynamodb = boto3.resource('dynamodb', region_name='ap-southeast-2')

# Các bảng DynamoDB
customer_segments_table = dynamodb.Table('CustomerSegments')
city_segments_table = dynamodb.Table('CitySegments')
at_risk_customers_table = dynamodb.Table('AtRiskCustomers')

# Hàm chuyển đổi float sang Decimal
def float_to_decimal(data):
    if isinstance(data, list):
        return [float_to_decimal(item) for item in data]
    elif isinstance(data, dict):
        return {k: float_to_decimal(v) for k, v in data.items()}
    elif isinstance(data, float):
        return Decimal(str(data))  # Chuyển float sang Decimal
    else:
        return data

# Hàm lưu Spark DataFrame vào DynamoDB
def save_to_dynamodb(df, table, partition_key, sort_key=None):
    # Chuyển đổi Spark DataFrame thành JSON
    data = df.toJSON().collect()
    data_with_decimal = [float_to_decimal(json.loads(item)) for item in data]

    # Kiểm tra và thêm khóa chính
    for item in data_with_decimal:
        # Đổi tên cột nếu cần và đảm bảo khóa chính là chuỗi
        if "Membership Type" in item and partition_key == "MembershipType":
            item["MembershipType"] = item.pop("Membership Type")
        if "Customer ID" in item and partition_key == "CustomerID":
            item["CustomerID"] = str(item.pop("Customer ID"))  # Chuyển sang chuỗi
        if "City" in item and partition_key == "City":
            item["City"] = item.pop("City")

        # Kiểm tra Partition Key
        if partition_key not in item or item[partition_key] is None:
            raise ValueError(f"Missing partition key '{partition_key}' in item: {item}")

        # Kiểm tra Sort Key (nếu có)
        if sort_key and (sort_key not in item or item[sort_key] is None):
            raise ValueError(f"Missing sort key '{sort_key}' in item: {item}")

        # Ghi dữ liệu vào DynamoDB
        table.put_item(Item=item)

# Chuyển đổi Customer ID thành chuỗi trước khi lưu
at_risk_customers = at_risk_customers.withColumn("Customer ID", at_risk_customers["Customer ID"].cast("string"))

# Lưu Customer Segments
print("Đang lưu CustomerSegments...")
save_to_dynamodb(
    customer_segments,
    customer_segments_table,
    partition_key="MembershipType",
    sort_key="City"
)

# Lưu City Segments
print("Đang lưu CitySegments...")
save_to_dynamodb(
    city_segments,
    city_segments_table,
    partition_key="City"
)

# Lưu At-Risk Customers
print("Đang lưu AtRiskCustomers...")
save_to_dynamodb(
    at_risk_customers,
    at_risk_customers_table,
    partition_key="CustomerID"
)

print("Tất cả dữ liệu đã được lưu vào DynamoDB thành công!")


In [ ]:
!aws configure